# CSC324 Lecture 2

## Haskell Language
### Constructors
Everything in haskell is defined as variables. Even functions are defined as variables.

In [1]:
a = 2
a

2

In [2]:
diffSq x y = (x - y) * (x + y)
diffSq 4 3

7

#### Local variables 
There are multiple ways to define local variables in haskell. We can define a local variable by the `let` keyword. For example `let x = 3`.  
We can use the `in` keyword to specify the outcome of the result.  
`let y = 5 in y * 4`  
If we want to keep a placeholder in variables and define them later, then we can use `where`. 
```
let a = b + c 
    where
        b = 3
        c = 4
```
It's important to note that `let ... in` is an expression whereas `where` is part of a definition

### [Pattern matching](http://learnyouahaskell.com/syntax-in-functions)
The idea behind pattern matching is similar to if else case.  
If your function has cases for your input then you could consider doing the following:  
```
function case1 = case1_solution
function case2 = case2_solution
...
function general = function recursive_input
```

In [3]:
slowFactorial 0 = 1
slowFactorial n = n * slowFactorial (n - 1)

In [4]:
slowFactorial 0
slowFactorial 15

1

1307674368000

It is possible to use if else cases in Haskell but generally it's ugly and is not preferred. It's also not optimized well.

In [5]:
uglyFactorial n = 
    if n == 0
        then 1
        else n * uglyFactorial (n-1)

In [6]:
uglyFactorial 0
uglyFactorial 15

1

1307674368000

### General approach to Recursive Functions
Never try to unfold recursive calls or else your head might explode. Instead, try coming up with a function by constructing a proof by induction.  

For example, in the case of slowFactorial:  
#### Base case
We know that `slowFactorial 0` is supposed to be 1. So we can let that be a **special case** for our function  
`slowFactorial 0 = 1`
#### Induction Step
Suppose `slowFactorial` works for `n-1`. We want to show that it works for `n`.  
By definition, $n! = n * (n-1)!$. We already know what $n!$ is, therefore we can write `slowFactorial n = n * slowFactorial (n-1)`

#### Evaluation view

This is highly not recommended to use as you will tear up and probably drop the course  

```
Evaluation view (how a computer or an enslaved student runs code): Plug and chug:

  slowFactorial 3
→ 3 * slowFactorial (3 - 1)
→ 3 * slowFactorial 2
→ 3 * (2 * slowFactorial (2 - 1))
→ 3 * (2 * slowFactorial 1)
→ 3 * (2 * (1 * slowFactorial (1 - 1)))
→ 3 * (2 * (1 * slowFactorial 0))
→ 3 * (2 * (1 * 1))
→ 3 * (2 * 1)
→ 3 * 2
→ 6
```

### Enums
Enums in Haskell are algebraic data types. They aren't the same as enums defined in `C` in the sense that they are just integers. They are real enumarations

In [7]:
data Direction = North | East | South | West
    deriving (Eq, Show)
-- Show is used for printing (Similar to __repr__ in python)
-- Eq is used for equality (Similar to __eq__ in python)
a = North
a

North

In [8]:
-- So for example, if I have another enum
data FakeDirection = Noorth | Eaast | Soouth | Weest
b = Noorth
a == b

<interactive>:1:6: error:
    • Couldn't match expected type ‘Direction’ with actual type ‘FakeDirection’
    • In the second argument of ‘(==)’, namely ‘b’
      In the expression: a == b
      In an equation for ‘it’: it = a == b

These data constructors/enums can also be used in pattern matching.

In [9]:
bearing North = 0
bearing East = 90
bearing South = 180
bearing West = 270

bearing West

270

In [10]:
direction 0 = North
direction 90 = East
direction 180 = South
direction 270 = West
direction _ = error "unsupported bearing"

direction 90
direction 220

East

unsupported bearing
CallStack (from HasCallStack):
  error, called at <interactive>:5:15 in interactive:Ghci212

You can also have pattern matching in data constructors as well 

In [11]:
data Tetrastratan
    = Monarch
    | Lord String String Integer
    | Knight String
    | Peasant String
    deriving (Eq, Show)

In [12]:
ninthDukeOfSussex = Lord "Duke" "Sussex" 9

-- How to address a Tetrastratan:
addressTetra Monarch = "H.M. The Monarch"
addressTetra (Lord d t i) = "The " ++ show i ++ "th " ++ d ++ " of " ++ t
addressTetra (Knight n) = "Sir " ++ n
addressTetra (Peasant n) = n

addressTetra ninthDukeOfSussex

"The 9th Duke of Sussex"

### Recursive Data Structures
We can also define recursive data structures using enums. Consider a linked list for example. It has a head and the remaining is just a linked list

In [13]:
data MyIntegerList = INil | ICons Integer MyIntegerList
    deriving Show -- INil if the list is empty
                  -- ICons defines the LinkedList
                  -- where the first integer is head
                  -- and the rest (recursive) is a linkedlist
exampleMyIntegerList = ICons 4 (ICons (-10) INil)

In [14]:
myISum :: MyIntegerList -> Integer
myISum INil = 0
myISum (ICons x xs) = x + myISum xs

myISum exampleMyIntegerList -- (4 - 10)

-6

Here is a binary search tree of integers. It will have the BST property

In [15]:
data IntegerBST = IEmpty | INode IntegerBST Integer IntegerBST
    deriving Show
exampleIntegerBST = INode (INode IEmpty 3 IEmpty) 7 (INode IEmpty 10 IEmpty)

We will now write a function to insert an element to the tree. The main issue that we are not allowed to manipulate data structures in Haskell. In other words, data in Haskell is **immutable**.   Thus when we call insert, we have to actually return a new tree.  

Like previously, the best way to come up with recursive functions is to construct an induction proof:  
#### Base Case
When you insert an element to an empty tree, you just return a leaf with a single element. So that's not difficult.
`ibstInsert k IEmpty = INode IEmpty k IEmpty`

#### Induction Step
Suppose the input tree is of the form `Inode left (left subtree) k (key) right (right subtree)`  
We will use strong induction and assume that inserting in smaller subtrees results works well.  
Let us consider subcases  
##### Subcase 1: k < key
In this case, we have to insert the key into the left subtree. So we call ibstInsert k left and by IH we assume that works. Thus we return a tree with the new element inserted into the left subtree (`INode (ibstInsert k left) key right`)
##### Subcase 2: k > key
In this case, we have to insert the key into the right subtree. This is similar to subcase1 (`INode left key (ibstInsert k right)`)
##### Subcase 3: k = key
In this case, we will assume that we just return the same tree.

In [16]:
ibstInsert :: Integer -> IntegerBST -> IntegerBST
ibstInsert k IEmpty =
    INode IEmpty k IEmpty

ibstInsert k inp@(INode left key right) -- "as-pattern", "inp as (Node left key right)"
    | k < key = INode (ibstInsert k left) key right
    | k > key = INode left key (ibstInsert k right)
    | otherwise = inp

ibstInsert 1 exampleIntegerBST

INode (INode (INode IEmpty 1 IEmpty) 3 IEmpty) 7 (INode IEmpty 10 IEmpty)

Note above that `inp@(INode left key right)` serves as a replacement for anywhere where inp is added in the function (Similar to function macros in `C`). So in the case of `otherwise`, `inp` will be replaced by `INode left key right`